High level overview of what's happening here:


1.   I'm using Open AI's Whisper model to seperate audio into segments and generate transcripts.
2.   I'm then generating speaker embeddings for each segments.
3.   Then I'm using agglomerative clustering on the embeddings to identify the speaker for each segment.   



In [ ]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))
#/content/3rd_Grade_Main_Idea_Lesson.mp3

Saving 4th_Grade_Science.mp3 to 4th_Grade_Science.mp3


In [ ]:
num_speakers = 2 #@param {type:"integer"}

language = 'English' #@param ['any', 'English']

model_size = 'medium' #@param ['tiny', 'base', 'small', 'medium', 'large']


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'


In [ ]:
# path = "mono_audio.wav"

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

In [ ]:
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

In [ ]:
import librosa
import soundfile as sf

# Convert audio to mono
y, sr = librosa.load(path, sr=None, mono=True)
sf.write("mono_audio.wav", y, sr)

# Update path to mono audio file
path = "mono_audio.wav"


In [ ]:
# #noise reduction
# import librosa
# y, sr = librosa.load(path, sr=None)
# y_denoised = librosa.effects.remix(y)


TypeError: remix() missing 1 required positional argument: 'intervals'

In [ ]:
# #normalization
# import soundfile as sf
# sf.write("normalized_audio.wav", librosa.util.normalize(y), sr)


In [ ]:
# path = "normalized_audio.wav"

In [ ]:
model = whisper.load_model(model_size)

100%|█████████████████████████████████████| 1.42G/1.42G [00:23<00:00, 65.2MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [ ]:
result = model.transcribe(path)
segments = result["segments"]

In [ ]:
print(result)

{'text': " Today, since we've been observing soil and plants, we've been observing those. We are going to, today, observe something else for our terrarium. And some of you have noticed what that was. We are going to be observing and exploring crickets. And what it has to do and going to have to do with our food chain. We're developing our food chain. Please turn to page 19 in your journals. Now, what we're going to do is we're going to get our materials managers. And I've got a tray of objects. And I'm going to show you those objects first. I'm going to, you're going to get, materials managers are going to get a tray. And on the tray, you've got a container. Inside the container is a cricket. Now, there might be one or two containers with more than one cricket. You're also going to get four magnifying glasses. Okay? Once this is on your table, the materials managers can come get the terrarium as well. Okay? So one terrarium per group. Whoa! Looks great. Wow! A real live cricket. Wow! L

In [ ]:
print(segments)

[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 13.8, 'text': " We're going to learn today about the main idea of a story and how to find the supporting", 'tokens': [50364, 492, 434, 516, 281, 1466, 965, 466, 264, 2135, 1558, 295, 257, 1657, 293, 577, 281, 915, 264, 7231, 51054], 'temperature': 0.0, 'avg_logprob': -0.21576957269148392, 'compression_ratio': 1.7880434782608696, 'no_speech_prob': 0.23309707641601562}, {'id': 1, 'seek': 0, 'start': 13.8, 'end': 14.8, 'text': ' details.', 'tokens': [51054, 4365, 13, 51104], 'temperature': 0.0, 'avg_logprob': -0.21576957269148392, 'compression_ratio': 1.7880434782608696, 'no_speech_prob': 0.23309707641601562}, {'id': 2, 'seek': 0, 'start': 14.8, 'end': 16.84, 'text': ' But you know how I am.', 'tokens': [51104, 583, 291, 458, 577, 286, 669, 13, 51206], 'temperature': 0.0, 'avg_logprob': -0.21576957269148392, 'compression_ratio': 1.7880434782608696, 'no_speech_prob': 0.23309707641601562}, {'id': 3, 'seek': 0, 'start': 16.84, 'end': 18.76, 'text': 

In [ ]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [ ]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [ ]:
#overlapping
# audio = Audio()
# def segment_embedding(segment):
#     start = max(0, segment["start"] - 0.5)  # Add a buffer
#     end = min(duration, segment["end"] + 0.5)
#     clip = Segment(start, end)
#     waveform, _ = audio.crop(path, clip)
#     return embedding_model(waveform[None])

In [ ]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [ ]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [ ]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("4th_Grade_Science_binary.txt", "w")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()

In [ ]:
# from sklearn.cluster import SpectralClustering
# # clustering = SpectralClustering(n_clusters=num_speakers, affinity='nearest_neighbors').fit(embeddings)

# # Spectral Clustering
# clustering = SpectralClustering(
#     n_clusters=num_speakers,        # Number of speakers (adjust as needed)
#     affinity='nearest_neighbors',   # Use nearest neighbors for similarity
#     n_neighbors=10,                 # Number of neighbors (tune based on dataset)
#     random_state=42                 # For reproducibility
# ).fit(embeddings)

# # Assign speaker labels from clustering
# labels = clustering.labels_

# # Update speaker labels in segments
# for i in range(len(segments)):
#     segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

# # Write the transcript with speaker diarization
# def time(secs):
#     return datetime.timedelta(seconds=round(secs))

# with open("transcript_5.txt", "w") as f:
#     for i, segment in enumerate(segments):
#         if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
#             f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
#         f.write(segment["text"][1:] + ' ')
